https://www.kaggle.com/ruchi798/movies-on-netflix-prime-video-hulu-and-disney

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rating_db = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")

In [3]:
rating_db.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [4]:
def clacla(x):
    try:
        return x.split(",")
    except:
        return []

In [5]:
from collections import Counter

# Não mexer:
#sorted(list(Counter(rating_db["Directors"].apply(clacla).apply(pd.Series).unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: x[1])

sorted(list(Counter(rating_db["Directors"].apply(clacla).apply(pd.Series).unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

[('Jay Chapman', 36),
 ('Joseph Kane', 31),
 ('Cheh Chang', 29),
 ('Jim Wynorski', 23),
 ('William Beaudine', 23),
 ('Sam Newfield', 22),
 ('Jay Karas', 21),
 ('Raúl Campos', 21),
 ('Jan Suter', 21),
 ('David DeCoteau', 21),
 ('Marcus Raboy', 18),
 ('Fred Olen Ray', 18),
 ('Scott L. Montoya', 17),
 ('William Witney', 17),
 ('Mark Atkins', 16),
 ('Lesley Selander', 16),
 ('Manny Rodriguez', 15),
 ('Paul Hoen', 15),
 ('Robert Stevenson', 14),
 ('William Nigh', 14),
 ('Philip Gardiner', 13),
 ('Mario Bava', 13),
 ('Roger Corman', 13),
 ('Robert N. Bradbury', 13),
 ('Ryan Polito', 12),
 ('Chia-Liang Liu', 12),
 ('Priyadarshan', 12),
 ('Shannon Hartman', 12),
 ('Frank Capra', 12),
 ('Stuart Gillard', 12),
 ('James Algar', 12),
 ('Leslie Small', 11),
 ('Lance Bangs', 11),
 ('Takashi Miike', 11),
 ('Rocco Urbisci', 11),
 ('Edward L. Cahn', 11),
 ('Steven Spielberg', 10),
 ('Ram Gopal Varma', 10),
 ('Richard Rich', 10),
 ('Uwe Boll', 10),
 ('Yash Chopra', 10),
 ('Harry L. Fraser', 10),
 ('Ron 

In [6]:
def sorted_bom(x):
    return sorted(list(Counter(rating_db[x].apply(clacla).apply(pd.Series).unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

In [7]:
%%time
#Conversão dos diretores com apenas um filme para uma categoria "outros"
directors_per_movie = rating_db .Directors.apply(clacla).apply(pd.Series)
all_directors_contributions = sorted_bom("Directors")

#Diretores a serem trocados
others = []

#Identificação dos diretores com apenas uma contribuição
for director,contributions in all_directors_contributions:
    if int(contributions) == 1:
        others.append(director)

#Listando somente diretores com mais de 2 contribuições
official_directors = directors_per_movie.replace(others, "Outros")

Wall time: 4min


In [9]:
directors = sorted(list(Counter(official_directors.unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

In [12]:
from sklearn.preprocessing import OneHotEncoder
#Hot-Encoder "manual" 

#DataFrame coluna de diretores
dir_df = pd.DataFrame([tpl[0] for tpl in directors])

#Encoder inicial para diretores
directors_encoder = OneHotEncoder(sparse=False)
dir_hot = directors_encoder.fit_transform(dir_df)


#Dicionário para relacionar diretor com seu respectivo código
dir_dict = dict(zip([tpl[0] for tpl in directors],dir_hot))
directors_transformed = official_directors.copy()

#Substituição dos diretores pelos respectivos códigos
for column in range(len(directors_transformed.columns)):
    for line in range(len(directors_transformed)):
        dct = directors_transformed.loc[line,column]
        try:
            directors_transformed.loc[line,column] = dir_dict[dct]
        except:
            #Caso seja uma linha cujo o diretor não é conhecido (nan), passa
            pass

#Soma-se os códigos para ter um único vetor contendo os códigos dos diretores
directors_transformed = directors_transformed.sum(axis=1).values        

#Substituição do vetor código no lugar dos nomes dos diretores
dir_final_convertion = []
for i in range(len(directors_transformed )):
    try:
        #Há casos em que a categoria "outros" aparece mais de uma vez. Logo, converte-se o valor para 1
        dir_final_convertion.append([*map(lambda x: int(x > 0), directors_transformed[i])])
    except:
        dir_final_convertion.append(np.nan)
rating_db_2 = rating_db.copy()
rating_db_2['Directors'] = dir_final_convertion
rating_db_2.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Western,"Italy,Spain,West Germany",Italian,161.0


In [27]:
list(rating_db_2.loc[1,'Directors']).index(1), list(rating_db_2.loc[1,'Directors'])[1625]

(1564, 1)

In [28]:
sum(list(rating_db_2.loc[1,'Directors']))

2

In [29]:
rating_db.loc[1,'Directors']

'Lana Wachowski,Lilly Wachowski'

In [30]:
list(dir_dict['Lana Wachowski']).index(1), list(dir_dict['Lilly Wachowski']).index(1)

(1564, 1625)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
country = rating_db[["Country"]]
genres = sorted_bom("Genres")

# pipe_sorted = Pipeline([
#     ("Country", OneHotEncoder(sparse = False),
#     ("Genres", OneHotEncoder(sparse = False),
#     ("Director", OneHotEncoder)
# ])

encoder = OneHotEncoder(categories="auto")
rating_db_1hot = encoder.fit_transform(genres)

In [10]:
rating_db_1hot = encoder.fit_transform(genres)

In [11]:
print(rating_db_1hot.toarray()[:5])

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0.]]


In [12]:
cat_pipeline = Pipeline([
    ("cat_encoder", OneHotEncoder(sparse=False)),
])
housing_cat_tr = cat_pipeline.fit_transform(rating_db)

In [15]:
cat_attribs = ["Country"]
genres = sorted_bom("Genres")
directors = sorted(list(Counter(official_directors.unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

full_pipeline = ColumnTransformer([
    ("cat", OneHotEncoder(sparse = False), cat_attribs),
    ("aaa_n_sei", OneHotEncoder(sparse = False), genres),
])

In [18]:
genres

[('Drama', 7227),
 ('Comedy', 4637),
 ('Thriller', 3354),
 ('Action', 2683),
 ('Documentary', 2546),
 ('Romance', 2359),
 ('Horror', 2201),
 ('Crime', 1833),
 ('Adventure', 1778),
 ('Family', 1514),
 ('Mystery', 1274),
 ('Sci-Fi', 1228),
 ('Fantasy', 1089),
 ('Biography', 821),
 ('History', 715),
 ('Animation', 711),
 ('Music', 667),
 ('War', 539),
 ('Western', 499),
 ('Sport', 404),
 ('Musical', 401),
 ('Short', 386),
 ('News', 105),
 ('Film-Noir', 79),
 ('Reality-TV', 22),
 ('Talk-Show', 19),
 ('Game-Show', 7)]

In [16]:
rating_db_prepared = full_pipeline.fit_transform(rating_db)

In [19]:
rating_db_prepared[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Exclui colunas não categóricas
rating_db_v2 = rating_db.drop(columns = ["ID", "Title", "IMDb", "Rotten Tomatoes", "Country", "Genres", "Directors", "Type"])
# Language, Age

In [ ]:
rating_db_v2